In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install scanpy --quiet

     |████████████████████████████████| 2.0 MB 4.3 MB/s 
     |████████████████████████████████| 9.4 MB 49.4 MB/s 
     |████████████████████████████████| 96 kB 5.7 MB/s 
     |████████████████████████████████| 88 kB 6.9 MB/s 
     |████████████████████████████████| 965 kB 74.9 MB/s 
     |████████████████████████████████| 295 kB 57.9 MB/s 
     |████████████████████████████████| 1.1 MB 67.7 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 


In [34]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import copy
import h5py
import torch
from scipy import sparse
from collections import Counter
from torch.utils.data import Dataset, DataLoader

In [28]:
from argparse import Namespace
config = Namespace(
    BATCH_SIZE = 500,
    NUM_WORKERS = 4,
    N_GENES = 13431,
    N_PEAKS = 116465,
    N_CHANNELS = 32,
    MAX_SEQ_LEN_GEX = 1500,
    MAX_SEQ_LEN_ATAC = 15000
)

In [4]:
def get_chr_index(adata_atac):
  r"""
  Output row indices for each chromosome for each chromosome
  Parameters
  ----------
  adata_atac
      annData for ATAC
  Returns
  -------
  chr_index
      Dictionary of indices for each chromosome
  """
  row_name = adata_atac.var.index
  chr_name = [c.split("-")[0] for c in row_name]
  lst = np.unique(chr_name) # names for chromosome

  chr_index = dict()
  for i in range(len(lst)):
    index = [a for a, l in enumerate(chr_name) if l == lst[i]]
    if lst[i] not in chr_index:
      chr_index[lst[i]]=index

  return chr_index

In [5]:
# Test for split_chr()
# adata_atac = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/ATAC_processed.h5ad")
# idx = split_chr(adata_atac)
# [len(idx[x]) for x in idx.keys()] # number of ATAC peaks in chromosomes

In [47]:
class MultiomeDataset(Dataset):
    def __init__(
        self, csr_gex, csr_atac
    ):
        super().__init__()
        
        self.csr_gex = csr_gex
        self.csr_atac = csr_atac
    
    def __len__(self):
        return self.csr_gex.shape[0]
    
    def __getitem__(self, index: int):
        x_gex = torch.tensor(self.csr_gex[index,:].todense())
        x_atac = torch.tensor(self.csr_atac[index,:].todense())
        return {'gex':x_gex, 'atac':x_atac}
  
  

In [51]:
def get_dataloaders(gex_train, atac_train, gex_val, atac_val):
    
    # mod2_train = mod2_train.iloc[sol_train.values.argmax(1)]
    # mod2_test = mod2_test.iloc[sol_test.values.argmax(1)]
    
    dataset_train = MultiomeDataset(gex_train, atac_train)
    data_train = DataLoader(dataset_train, config.BATCH_SIZE, shuffle = True, num_workers = config.NUM_WORKERS)
    
    dataset_val = MultiomeDataset(gex_val, atac_val)
    data_val = DataLoader(dataset_val, config.BATCH_SIZE, shuffle = False, num_workers = config.NUM_WORKERS)
    
    return data_train, data_val

In [10]:
# csr_gex = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/GEX_processed.h5ad").layers['log_norm']
# csr_atac = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/ATAC_processed.h5ad").layers['log_norm']

In [11]:
# batch = ad.read_h5ad("drive/MyDrive/Colab_Notebooks/CPSC532S/final_project/data/GEX_processed.h5ad").obs['batch']

In [20]:
# batch = list(batch)
# train_id = [a for a, l in enumerate(batch) if l not in ['s2d4','s1d1']]
# val_id =  [a for a, l in enumerate(batch) if l == 's1d1']
# test_id = [a for a, l in enumerate(batch) if l == 's2d4']

In [22]:
# gex_train = csr_gex[train_id,:]
# atac_train = csr_atac[train_id,:]

# gex_val = csr_gex[val_id,:]
# atac_val = csr_atac[val_id,:]

In [49]:
# data_train, data_val = get_dataloaders(gex_train, atac_train, gex_val, atac_val)


In [50]:
# for i,data in enumerate(data_train):
#   if i == 0:
#     print(data['gex'].size())
#   else:
#     break

torch.Size([500, 1, 13431])
